In [1]:
import caffe
caffe.set_mode_gpu()
caffe.set_device(3)

In [5]:
net = caffe.Net('data/models/airStern_patchDetection/val.prototxt', 'data/models/airStern_patchDetection/snapshots/train_iter_4000.caffemodel', caffe.TEST)
params = ['fc6_', 'fc7_']
fc_params = {pr: (net.params[pr][0].data, net.params[pr][1].data) for pr in params}

for fc in params:
    print('{} weights are {} dimensional and biases are {} dimensional'.format(fc, fc_params[fc][0].shape, fc_params[fc][1].shape))

fc6_ weights are (4096, 100352) dimensional and biases are (4096,) dimensional
fc7_ weights are (4096, 4096) dimensional and biases are (4096,) dimensional


In [11]:
# Load the fully convolutional network to transplant the parameters.
net_full_conv = caffe.Net('data/models/airStern_patchDetection_FCN/train.prototxt', 
                          'data/models/airStern_patchDetection/snapshots/train_iter_4000.caffemodel',
                          caffe.TEST)
params_full_conv = ['fc6_conv', 'fc7_conv']
# conv_params = {name: (weights, biases)}
conv_params = {pr: (net_full_conv.params[pr][0].data, net_full_conv.params[pr][1].data) for pr in params_full_conv}

for conv in params_full_conv:
    print('{} weights are {} dimensional and biases are {} dimensional'.format(conv, conv_params[conv][0].shape, conv_params[conv][1].shape))

fc6_conv weights are (4096, 512, 7, 7) dimensional and biases are (4096,) dimensional
fc7_conv weights are (4096, 4096, 1, 1) dimensional and biases are (4096,) dimensional


In [12]:
for pr, pr_conv in zip(params, params_full_conv):
    conv_params[pr_conv][0].flat = fc_params[pr][0].flat  # flat unrolls the arrays
    conv_params[pr_conv][1][...] = fc_params[pr][1]


In [13]:
net_full_conv.save('data/models/airStern_patchDetection/snapshots/fcn_train_iter_4000.caffemodel')